# FLO CLTV Prediction (BG-NBD and Gamma Gamma Model)

In [63]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from sklearn.preprocessing import MinMaxScaler

## Task 1: Data Preprocessing

In [64]:
path = "./flo_data_20k.csv"
flo_dataframe = pd.read_csv(path)
df = flo_dataframe.copy()
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]


In [65]:
print(df.describe([0, 0.01, 0.50, 0.95, 0.99, 1]).T)

                                     count   mean    std   min    0%    1%    50%     95%     99%     100%      max
order_num_total_ever_online       19945.00   3.11   4.23  1.00  1.00  1.00   2.00   10.00   20.00   200.00   200.00
order_num_total_ever_offline      19945.00   1.91   2.06  1.00  1.00  1.00   1.00    4.00    7.00   109.00   109.00
customer_value_total_ever_offline 19945.00 253.92 301.53 10.00 10.00 19.99 179.98  694.22 1219.95 18119.14 18119.14
customer_value_total_ever_online  19945.00 497.32 832.60 12.99 12.99 39.99 286.46 1556.73 3143.81 45220.13 45220.13


In [66]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [67]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit)

In [68]:
replace_with_thresholds(df,"order_num_total_ever_offline")
replace_with_thresholds(df,"order_num_total_ever_online")

In [69]:
print(df.describe([0, 0.01, 0.50, 0.95, 0.99, 1]).T)

                                     count   mean    std   min    0%    1%    50%     95%     99%     100%      max
order_num_total_ever_online       19945.00   3.09   3.81  1.00  1.00  1.00   2.00   10.00   20.00    48.00    48.00
order_num_total_ever_offline      19945.00   1.89   1.43  1.00  1.00  1.00   1.00    4.00    7.00    16.00    16.00
customer_value_total_ever_offline 19945.00 253.92 301.53 10.00 10.00 19.99 179.98  694.22 1219.95 18119.14 18119.14
customer_value_total_ever_online  19945.00 497.32 832.60 12.99 12.99 39.99 286.46 1556.73 3143.81 45220.13 45220.13


In [70]:
df["order_num_total"] = df["order_num_total_ever_offline"] + df["order_num_total_ever_online"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [71]:
df.head(2)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN],5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.00,2013.55


In [72]:
print(df.describe([0, 0.01, 0.50, 0.95, 0.99, 1]).T)

                                     count   mean    std   min    0%     1%    50%     95%     99%     100%      max
order_num_total_ever_online       19945.00   3.09   3.81  1.00  1.00   1.00   2.00   10.00   20.00    48.00    48.00
order_num_total_ever_offline      19945.00   1.89   1.43  1.00  1.00   1.00   1.00    4.00    7.00    16.00    16.00
customer_value_total_ever_offline 19945.00 253.92 301.53 10.00 10.00  19.99 179.98  694.22 1219.95 18119.14 18119.14
customer_value_total_ever_online  19945.00 497.32 832.60 12.99 12.99  39.99 286.46 1556.73 3143.81 45220.13 45220.13
order_num_total                   19945.00   4.98   4.12  2.00  2.00   2.00   4.00   12.00   22.00    57.00    57.00
customer_value_total              19945.00 751.24 895.40 44.98 44.98 113.98 545.27 1921.92 3606.36 45905.10 45905.10


In [73]:
df.dtypes

master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
order_num_total                      float64
customer_value_total                 float64
dtype: object

In [74]:
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [75]:
df.dtypes

master_id                                    object
order_channel                                object
last_order_channel                           object
first_order_date                     datetime64[ns]
last_order_date                      datetime64[ns]
last_order_date_online               datetime64[ns]
last_order_date_offline              datetime64[ns]
order_num_total_ever_online                 float64
order_num_total_ever_offline                float64
customer_value_total_ever_offline           float64
customer_value_total_ever_online            float64
interested_in_categories_12                  object
order_num_total                             float64
customer_value_total                        float64
dtype: object

## Task2: CLTV Data Structure

In [76]:
last_date = max(df["last_order_date"])
last_date

Timestamp('2021-05-30 00:00:00')

In [77]:
# Decide to set analysis date after 2 days the last_date that has been done shopping by any customer
analysis_date = dt.datetime(2021, 6, 1)
analysis_date

datetime.datetime(2021, 6, 1, 0, 0)

In [78]:
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] =  ((df["last_order_date"] - df["first_order_date"])/7).astype('timedelta64[W]')
cltv_df["T_weekly"] =  ((analysis_date - df["first_order_date"])/7).astype('timedelta64[W]')
cltv_df["frequency"] =  df["order_num_total"]
cltv_df["monetary_cltv_avg"] =  df["customer_value_total"] / df["order_num_total"]

In [79]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency_cltv_weekly,19945.00,13.12,10.66,0.00,7.00,10.00,15.00,61.00
T_weekly,19945.00,15.87,10.69,0.00,10.00,13.00,17.00,62.00
frequency,19945.00,4.98,4.12,2.00,3.00,4.00,6.00,57.00
monetary_cltv_avg,19945.00,152.61,83.92,22.49,103.65,136.91,182.49,5176.59


In [80]:
cltv_df = cltv_df[(cltv_df['recency_cltv_weekly'] > 0)]
cltv_df = cltv_df[(cltv_df['T_weekly'] > 0)]

In [81]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency_cltv_weekly,19347.00,13.53,10.56,1.00,7.00,11.00,15.00,61.00
T_weekly,19347.00,16.27,10.59,1.00,10.00,13.00,17.00,62.00
frequency,19347.00,5.05,4.16,2.00,3.00,4.00,6.00,57.00
monetary_cltv_avg,19347.00,151.42,72.56,22.49,103.49,136.46,181.48,1106.47


In [82]:
cltv_df.head(8)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,2.00,4.00,5.00,187.87
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,29.00,32.00,21.00,95.88
2,69b69676-1a40-11ea-941b-000d3a38a36f,7.00,11.00,5.00,117.06
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,11.00,13.00,2.00,104.99
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,17.00,18.00,3.00,66.95
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,4.00,9.00,4.00,93.98
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,1.00,7.00,2.00,81.81
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,8.00,10.00,5.00,210.94


## Task3: Create BG-NBD and Gamma Gamma Model to calculate CLTV 

In [83]:
# BG-NBD
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19347 subjects, a: 0.00, alpha: 10.55, b: 0.00, r: 3.62>

In [84]:
cltv_df["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4*3,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency_cltv_weekly'],
                                                        cltv_df['T_weekly'])

In [85]:
cltv_df.head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,2.00,4.00,5.00,187.87,7.11
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,29.00,32.00,21.00,95.88,6.94
2,69b69676-1a40-11ea-941b-000d3a38a36f,7.00,11.00,5.00,117.06,4.80
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,11.00,13.00,2.00,104.99,2.86
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,17.00,18.00,3.00,66.95,2.78
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,4.00,9.00,4.00,93.98,4.68
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,1.00,7.00,2.00,81.81,3.84
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,8.00,10.00,5.00,210.94,5.03
9,1143f032-440d-11ea-8b43-000d3a38a36f,8.00,13.00,2.00,82.98,2.86
11,c2e15af2-9eed-11e9-9897-000d3a38a36f,43.00,44.00,20.00,143.08,5.20


In [86]:
cltv_df["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4*6,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency_cltv_weekly'],
                                                        cltv_df['T_weekly'])

In [87]:
cltv_df.head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,2.00,4.00,5.00,187.87,7.11,14.22
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,29.00,32.00,21.00,95.88,6.94,13.89
2,69b69676-1a40-11ea-941b-000d3a38a36f,7.00,11.00,5.00,117.06,4.80,9.60
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,11.00,13.00,2.00,104.99,2.86,5.73
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,17.00,18.00,3.00,66.95,2.78,5.57
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,4.00,9.00,4.00,93.98,4.68,9.36
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,1.00,7.00,2.00,81.81,3.84,7.69
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,8.00,10.00,5.00,210.94,5.03,10.07
9,1143f032-440d-11ea-8b43-000d3a38a36f,8.00,13.00,2.00,82.98,2.86,5.73
11,c2e15af2-9eed-11e9-9897-000d3a38a36f,43.00,44.00,20.00,143.08,5.20,10.39


In [88]:
cltv_df.columns

Index(['customer_id', 'recency_cltv_weekly', 'T_weekly', 'frequency', 'monetary_cltv_avg', 'exp_sales_3_month', 'exp_sales_6_month'], dtype='object')

In [89]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg']) 

<lifetimes.GammaGammaFitter: fitted with 19347 subjects, p: 4.15, q: 0.47, v: 4.08>

!! NOTE: If you don not have integer frequency values this model will throw an exception about that. So, be careful.

In [90]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],cltv_df['monetary_cltv_avg'])

In [91]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,2.00,4.00,5.00,187.87,7.11,14.22,193.62
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,29.00,32.00,21.00,95.88,6.94,13.89,96.66
2,69b69676-1a40-11ea-941b-000d3a38a36f,7.00,11.00,5.00,117.06,4.80,9.60,120.96
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,11.00,13.00,2.00,104.99,2.86,5.73,114.31
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,17.00,18.00,3.00,66.95,2.78,5.57,71.34


#### 6 months CLTV 

In [92]:
cltv_df["cltv_6months"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6,  # 6 months
                                   freq="W",  # T value's frequency information: Weekly
                                   )

In [93]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv_6months
0,cc294636-19f0-11eb-8d74-000d3a38a36f,2.00,4.00,5.00,187.87,7.11,14.22,193.62,2889.11
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,29.00,32.00,21.00,95.88,6.94,13.89,96.66,1408.52
2,69b69676-1a40-11ea-941b-000d3a38a36f,7.00,11.00,5.00,117.06,4.80,9.60,120.96,1218.69
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,11.00,13.00,2.00,104.99,2.86,5.73,114.31,687.24
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,17.00,18.00,3.00,66.95,2.78,5.57,71.34,416.72


In [94]:
# First 20 customer who has the highest 6 months cltv values
cltv_df.sort_values("cltv_6months",ascending=False).head(20)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv_6months
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,19.00,21.00,49.00,751.39,20.01,40.03,753.44,31642.39
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,8.00,9.00,52.00,354.68,34.14,68.28,355.63,25476.37
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,56.00,56.00,50.00,918.10,9.67,19.34,920.53,18676.89
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,10.00,10.00,16.00,622.08,11.46,22.91,627.33,15082.77
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,54.00,55.00,49.00,692.21,9.63,19.27,694.09,14030.88
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,9.00,13.00,18.00,576.86,11.02,22.03,581.20,13436.48
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,14.00,18.00,20.00,605.16,9.93,19.86,609.24,12692.96
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,7.00,14.00,7.00,1106.47,5.19,10.38,1127.57,12285.11
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,1.00,1.00,17.00,259.87,21.42,42.84,262.07,11780.86
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,1.00,1.00,2.00,862.69,5.84,11.68,923.56,11319.98


## Task 4: Segmentation of the CLTV values

In [95]:
cltv_df["segment"] = pd.qcut(cltv_df["cltv_6months"], 4, labels=["D", "C", "B", "A"])

In [99]:
cltv_df.sort_values("cltv_6months",ascending=False).head(5)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv_6months,segment
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,19.00,21.00,49.00,751.39,20.01,40.03,753.44,31642.39,A
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,8.00,9.00,52.00,354.68,34.14,68.28,355.63,25476.37,A
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,56.00,56.00,50.00,918.10,9.67,19.34,920.53,18676.89,A
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,10.00,10.00,16.00,622.08,11.46,22.91,627.33,15082.77,A
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,54.00,55.00,49.00,692.21,9.63,19.27,694.09,14030.88,A


In [100]:
cltv_df.sort_values("cltv_6months").head(5)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv_6months,segment
1710,08b95b62-a987-11e9-a2fc-000d3a38a36f,32.00,33.00,2.00,22.49,1.55,3.10,26.20,85.18,D
27,c1f8f878-9f35-11e9-9897-000d3a38a36f,57.00,59.00,4.00,32.72,1.32,2.63,34.85,96.18,D
11232,f486e45e-a691-11e9-a2fc-000d3a38a36f,50.00,54.00,2.00,45.24,1.05,2.09,50.50,110.76,D
6963,7e0928c6-a6e3-11e9-a2fc-000d3a38a36f,48.00,50.00,4.00,32.94,1.51,3.02,35.07,111.18,D
2641,f7b3612e-a6ce-11e9-a2fc-000d3a38a36f,47.00,51.00,4.00,34.99,1.49,2.97,37.20,116.00,D


In [101]:
cltv_df.groupby("segment").agg(
    {"count", "mean", "sum"})

recency_cltv_weekly              T_weekly             frequency            monetary_cltv_avg              exp_sales_3_month            exp_sales_6_month             exp_average_value              cltv_6months              
                        sum  mean count       sum  mean count       sum mean count               sum   mean count               sum mean count               sum  mean count               sum   mean count          sum    mean count
segment                                                                                                                                                                                                                               
D                  95644.00 19.77  4837 111702.00 23.09  4837  18341.00 3.79  4837         451665.48  93.38  4837          14014.73 2.90  4837          28029.46  5.79  4837         478342.19  98.89  4837   2756552.33  569.89  4837
C                  62819.00 12.99  4837  76770.00 15.87  4837  21336.00 4.41  4837         607255.33 125.54  4837          18113.07 3.74  4837          36226.15  7.49  4837         638319.59 131.97  4837   4755560.20  983.16  4837
B                  56177.00 11.62  4836  68848.00 14.24  4836  25015.00 5.17  4836         772858.39 159.81  4836          20762.61 4.29  4836          41525.22  8.59  4836         807664.79 167.01  4836   6850726.10 1416.61  4836
A                  47131.00  9.74  4837  57474.00 11.88  4837  33047.00 6.83  4837        1097654.33 226.93  4837          26819.55 5.54  4837          53639.09 11.09  4837        1140009.61 235.69  4837  12458404.41 2575.65  4837

**RESULT**
* I would recommend FLO to focus on C and B segments. They are very similar to each other and they have the potential to grow like A segment. To focus on C and B segments would be benefical for FLO in 6 months.

## Bonus Task: Functonize All Process

In [102]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit
    
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit)

In [112]:
def create_cltv(dataframe,discount=0.01,month=6):
    # Preprocessing
    threshold_columns = ["order_num_total_ever_online","order_num_total_ever_offline","customer_value_total_ever_offline","customer_value_total_ever_online"]
    for col in threshold_columns:
        replace_with_thresholds(dataframe,col)
    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    dataframe = dataframe[~(dataframe["customer_value_total"]==0) | (dataframe["order_num_total"]==0)]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
    # CLTV Data Structure
    #dataframe["laste_order_date"].max()#result: 2021-05-30
    analysis_date = dt.datetime(2021,6,1)# YYYY,MM,DD
    cltv_df = pd.DataFrame()
    cltv_df["customer_id"] = dataframe["master_id"]
    cltv_df["recency"] = (dataframe["last_order_date"]-dataframe["first_order_date"]).astype("timedelta64[D]") / 7 # WEEKLY
    cltv_df["T"] = (analysis_date-dataframe["first_order_date"]).astype("timedelta64[D]") / 7 # WEEKLY
    cltv_df["frequency"] = dataframe["order_num_total"]
    cltv_df["monetary_avg"] = dataframe["customer_value_total"] / dataframe["order_num_total"]
    # All frequency values are greater than 1 so I am not controlling that value!!!!

    # BGN-BD Model
    try:
        bgf = BetaGeoFitter(penalizer_coef=0.001)
        bgf.fit(cltv_df["frequency"],cltv_df["recency"],cltv_df["T"])
        
    except:
        print("BG-NBD model could not fitted!")

    cltv_df["exp_sales_3_month"] = bgf.predict(4*3,cltv_df["frequency"],cltv_df["recency"],cltv_df["T"])
    cltv_df["exp_sales_6_month"] = bgf.predict(4*6,cltv_df["frequency"],cltv_df["recency"],cltv_df["T"])
    # GAMMA GAMMA Model
    try:
        ggf = GammaGammaFitter(penalizer_coef=0.01)
        ggf.fit(cltv_df["frequency"],cltv_df["monetary_avg"])
    except:
        print("Gamma Gamma model could not fitted!")

    cltv_df["exp_avg_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],cltv_df["monetary_avg"])
    cltv_df["cltv_"+str(month)] = ggf.customer_lifetime_value(bgf,
                                                       cltv_df["frequency"],cltv_df["recency"],cltv_df["T"],cltv_df["monetary_avg"],
                                                       time=month,freq="W",discount_rate=discount)
    cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv_"+str(month)],4,labels=["D","C","B","A"])

    return cltv_df



In [113]:
# TEST
result_cltv_df = create_cltv(flo_dataframe)

In [116]:
result_cltv_df.head()

,customer_id,recency,T,frequency,monetary_avg,exp_sales_3_month,exp_sales_6_month,exp_avg_value,cltv_6,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87,0.97,1.95,193.63,395.73,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88,0.98,1.97,96.67,199.43,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06,0.67,1.34,120.97,170.22,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98,0.70,1.40,67.32,98.95,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99,0.40,0.79,114.33,95.01,D


In [118]:
result_cltv_df.to_csv("flo_cltv_6_months.csv",index=False)